In [3]:
try:  
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox
    
    
class Alien(object):

    alive_image=None
    
    def __init__(self):
        self.id = None
        self.alive = True
    
    @classmethod
    def get_alive_image(cls):
        if cls.alive_image==None:
            cls.alive_image=tk.PhotoImage(file='alien.gif')
        return cls.alive_image
            
    def touched_by(self, canvas, projectile):
        None
        
    def install_in(self, canvas, x, y):
        self.id=canvas.create_image(x,y,image=Alien.get_alive_image())
        
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id,dx,dy)
        
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 2
        self.alien_y_delta = 15
        self.fleet_size = self.aliens_lines*self.aliens_columns
        self.aliens_fleet = [] 

    def install_in(self, canvas):
        x=50
        y=50
        for j in range(self.aliens_lines):
            for i in range(self.aliens_columns):
                alien=Alien()
                alien.install_in(canvas,x,y)
                self.aliens_fleet.append(alien)
                x=x+80
            x=x-80*self.aliens_columns
            y=y+70
            
    def move_in(self, canvas):
        max_w = int(canvas.cget("width"))
        x1, y1, x2, y2 = canvas.bbox(self.aliens_fleet[self.fleet_size-1].id)
        xa, ya, xb, yb = canvas.bbox(self.aliens_fleet[0].id)
        if x2 > max_w:
            self.alien_x_delta = -self.alien_x_delta
            for i in self.aliens_fleet:
                i.move_in(canvas,0,self.alien_y_delta)
        elif xa < 0:
            self.alien_x_delta = -self.alien_x_delta
            for i in self.aliens_fleet:
                i.move_in(canvas,0,self.alien_y_delta)
        for i in self.aliens_fleet:
            i.move_in(canvas,self.alien_x_delta,0)
        
        
    def manage_touched_aliens_by(self,canvas,defender):
        for i in defender.fired_bullets:
            for j in self.aliens_fleet:
                x1, y1, x2, y2 = canvas.bbox(i.id)
                xa, ya, xb, yb = canvas.bbox(j.id)
                if x1==xa and y1==ya :
                    j=None
                    i=None
                    self.defender.max_fired_bullets=self.defender.max_fired_bullets+1
                    
                    
    
class Defender(object):
    def __init__(self):
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8
        self.fired_bullets = []

    def install_in(self, canvas):
        x, y= 25, 660
        self.id = canvas.create_rectangle(x,y,x+self.width,y+self.height,fill="white")
        
    def move_in(self,canvas, dx):
        canvas.move(self.id, dx, 0)

    def fire(self, canvas):
        bullet=Bullet(self)                               #creation bullet
        self.fired_bullets.append(bullet)                 #ajout de bullet à la liste
        bullet.install_in(canvas)                         #installation du bullet dans le canvas
        
        
class Bullet(object):
    def __init__(self,shooter):
        self.radius = 5
        self.color = "red"
        self.speed = 20
        self.id = None
        self.shooter = shooter

    def install_in(self, canvas):
        x1, y1, x2, y2 = canvas.coords(self.shooter.id)                #recuperation coordonnées du defender
        self.id = canvas.create_oval(x1+5,y1-20,x1+5+2*self.radius,y1-20+2*self.radius,fill=self.color) #creation du bullet au dessus du defender dans le canvas

    def move_in(self, canvas):
            canvas.move(self.id, 0, -self.speed)
        
        
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.height =700
        self.width=1000
        #self.width = self.fleet.get_width()
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height, background='black')
        self.defender.install_in(self.canvas)
        self.canvas.pack()
        self.fleet.install_in(self.canvas)

    def animation(self):
        self.frame.winfo_toplevel().bind("<Key>", self.keypress)
        self.move_bullets()
        self.move_aliens_fleet()
        self.fleet.manage_touched_aliens_by(self.canvas,self.defender)
        self.canvas.after(300, self.animation)
        
    def keypress(self, event):
        x = 0
        x1, y1, x2, y2 = self.canvas.coords(self.defender.id)
        if event.keysym == 'Left' and x2>50: 
            x = -self.defender.move_delta 
        elif event.keysym == 'Right' and x2<965: 
            x = self.defender.move_delta
        self.defender.move_in(self.canvas, x)
        if event.keysym == 'space' and self.defender.max_fired_bullets>0:
            self.defender.max_fired_bullets=self.defender.max_fired_bullets-1       
            self.defender.fire(self.canvas)  
            
             
    def start_animation(self):
        self.canvas.after(10, self.animation())
        
    def move_bullets(self):
        for i in range(len(self.defender.fired_bullets)):           #pour tous les bullets de la liste
            if self.defender.fired_bullets[i]!=None:                #si il existe
                self.defender.fired_bullets[i].move_in(self.canvas)     #on bouge le bullet
                x1, y1, x2, y2 = self.canvas.coords(self.defender.fired_bullets[i].id)     #on récupère les coordonnées du bullet
                if (y2<0 and self.defender.max_fired_bullets<8) :                #si son ordonnée est <0 et qu'il y a moins de 8 bullets à l'écran
                    self.defender.fired_bullets[i]=None                #on suprime le bullet de la liste
                    self.defender.max_fired_bullets=self.defender.max_fired_bullets+1         #on recharge un coup
             
    def move_aliens_fleet(self):
        for i in range(10):
            self.fleet.move_in(self.canvas)
        
        
        
        
class SpaceInvaders(object):

    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both", expand=True)
        self.game=Game(self.frame)

    def play(self):
        self.game.start_animation()
        self.root.mainloop()
                        
                        
SpaceInvaders().play()
